In [193]:
# Requirements
import pandas as pd
import requests
from datetime import date
debug = True

In [194]:
# verify today
today = date.today()
today = today.strftime('%d/%m/%Y')
today

'09/07/2022'

In [195]:
# Read and write paths
INPUT_PATH = r'D:\Kike\Proyectos\Desarrollando\electrical_data\inputs'
INPUT_FILE = '/export_historico_format.csv'
OUTPUT_PATH = r'D:\Kike\Proyectos\Desarrollando\electrical_data\output'
OUTPUT_FILE = '/data_updated_'+str(today).replace('/','')+'.csv'

In [196]:
# API call
r = requests.get('https://api.preciodelaluz.org/v1/prices/all?zone=PCB')

In [197]:
# Functions
def replace_substrings(df):
    df = df.replace("'",'', regex=True)
    df = df.replace('{','', regex=True)
    df = df.replace('}','', regex=True)
    df['hour'] = df['hour'].str.replace("hour: ", '')
    df['day'] = df['day'].str.replace("date: ", '')
    df['is_cheap'] = df['is_cheap'].str.replace("is-cheap: ", '')
    df['under_avg'] = df['under_avg'].str.replace('is-under-avg: ', '')
    df['market'] = df['market'].str.replace('market: ', '')
    df['price'] = df['price'].str.replace('price: ', '')
    df['units'] = df['units'].str.replace('units: ', '')
    df = df.replace(' ','', regex=True)
    return df
    
def format_types(df):
    df['exec_day'] = today
    df['day'] = pd.to_datetime(df['day'],format='%d-%m-%Y') #'%Y-%d-%m'
    df['price'] = df['price'].astype('float')
    return df

In [198]:
df = pd.json_normalize(r.json(),max_level=0).T
df

,0
00-01,"{'date': '09-07-2022', 'hour': '00-01', 'is-ch..."
01-02,"{'date': '09-07-2022', 'hour': '01-02', 'is-ch..."
02-03,"{'date': '09-07-2022', 'hour': '02-03', 'is-ch..."
03-04,"{'date': '09-07-2022', 'hour': '03-04', 'is-ch..."
04-05,"{'date': '09-07-2022', 'hour': '04-05', 'is-ch..."
05-06,"{'date': '09-07-2022', 'hour': '05-06', 'is-ch..."
06-07,"{'date': '09-07-2022', 'hour': '06-07', 'is-ch..."
07-08,"{'date': '09-07-2022', 'hour': '07-08', 'is-ch..."
08-09,"{'date': '09-07-2022', 'hour': '08-09', 'is-ch..."
09-10,"{'date': '09-07-2022', 'hour': '09-10', 'is-ch..."


In [199]:
#JSON to first pandas df 
df = pd.json_normalize(r.json(),max_level=0).T
df = df.reset_index()
df = df.rename(columns={"index": "hour", 0 : "values"})
# dict to str
df['values']=df['values'].astype('str')
# split to values column
df[['day','hour','is_cheap','under_avg','market','price','units']] = df['values'].str.split(pat=",",expand=True)
# format columns items
df = replace_substrings(df)
df = format_types(df)
df = df.drop(['values'], axis=1)
# Variable api call day
new_day = df['day'][0].strftime('%Y-%m-%d')
df

,hour,day,is_cheap,under_avg,market,price,units,exec_day
0,00-01,2022-07-09,False,False,PVPC,346.26,€/Mwh,09/07/2022
1,01-02,2022-07-09,False,False,PVPC,351.17,€/Mwh,09/07/2022
2,02-03,2022-07-09,False,False,PVPC,360.78,€/Mwh,09/07/2022
3,03-04,2022-07-09,False,False,PVPC,377.19,€/Mwh,09/07/2022
4,04-05,2022-07-09,False,False,PVPC,390.59,€/Mwh,09/07/2022
5,05-06,2022-07-09,False,False,PVPC,396.92,€/Mwh,09/07/2022
6,06-07,2022-07-09,False,False,PVPC,413.83,€/Mwh,09/07/2022
7,07-08,2022-07-09,False,False,PVPC,392.79,€/Mwh,09/07/2022
8,08-09,2022-07-09,False,False,PVPC,356.30,€/Mwh,09/07/2022
9,09-10,2022-07-09,False,True,PVPC,305.55,€/Mwh,09/07/2022


In [200]:
# Read file all data
df_all = pd.read_csv(INPUT_PATH + INPUT_FILE, encoding='utf-8', delimiter=',')
df_all

,hour,day,price,is_cheap,under_avg,market,units
0,00-01,2022-01-01,204.51,NaN,NaN,PVPC,€/Mwh
1,01-02,2022-01-01,171.35,NaN,NaN,PVPC,€/Mwh
2,02-03,2022-01-01,172.70,NaN,NaN,PVPC,€/Mwh
3,03-04,2022-01-01,156.07,NaN,NaN,PVPC,€/Mwh
4,04-05,2022-01-01,159.08,NaN,NaN,PVPC,€/Mwh
...,...,...,...,...,...,...,...
4554,19-20,2022-07-09,286.16,NaN,NaN,PVPC,€/Mwh
4555,20-21,2022-07-09,329.35,NaN,NaN,PVPC,€/Mwh
4556,21-22,2022-07-09,350.58,NaN,NaN,PVPC,€/Mwh
4557,22-23,2022-07-09,354.28,NaN,NaN,PVPC,€/Mwh


In [201]:
# Secure filter by day
df_all['day'] = pd.to_datetime(df_all['day'])
df_all = df_all[df_all['day']!=new_day]
df_all

,hour,day,price,is_cheap,under_avg,market,units
0,00-01,2022-01-01,204.51,NaN,NaN,PVPC,€/Mwh
1,01-02,2022-01-01,171.35,NaN,NaN,PVPC,€/Mwh
2,02-03,2022-01-01,172.70,NaN,NaN,PVPC,€/Mwh
3,03-04,2022-01-01,156.07,NaN,NaN,PVPC,€/Mwh
4,04-05,2022-01-01,159.08,NaN,NaN,PVPC,€/Mwh
...,...,...,...,...,...,...,...
4530,19-20,2022-07-08,354.62,NaN,NaN,PVPC,€/Mwh
4531,20-21,2022-07-08,387.64,NaN,NaN,PVPC,€/Mwh
4532,21-22,2022-07-08,392.76,NaN,NaN,PVPC,€/Mwh
4533,22-23,2022-07-08,345.40,NaN,NaN,PVPC,€/Mwh


In [202]:
# Append to csv
new_df_all = df_all.append(df,ignore_index = True, verify_integrity = True)
if debug:
    new_df_all

In [203]:
#Export to csv
new_df_all.to_csv(OUTPUT_PATH + OUTPUT_FILE, index=False, encoding='utf-8')